In [26]:
# COURSE 4 - WEEK 3 - QUESTION 2

from bs4 import BeautifulSoup
import requests
import pandas as pd

# download url data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
urltext = requests.get(url).text
data = BeautifulSoup(urltext, 'lxml')

# creat a new Dataframe
columnlabel = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = columnlabel)

# loop through to find postcode, borough, neighborhood 
content = data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough != 'Not assigned\n']
#print(toronto)
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
# drop burroughs with none assigned
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

# group neighborhoods with like burroughs
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

# Read the cvs file and convert it to a dataframe
url='http://cocl.us/Geospatial_data'
df_pcodes=pd.read_csv(url)

#Change "Postal Code" to "Postalcode" for proper merger
df_pcodes.columns = ['Postalcode', 'Latitude', 'Longitude']

nhs=pd.merge(df2,df_pcodes, how='right', on = 'Postalcode')

#sort dataframe by postalcode
nhssort=nhs.sort_values('Postalcode')
nhssort.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
